In [1]:
!pip install geopy

In [2]:
from geopy.distance import great_circle
import pandas as pd
import folium

In [3]:
from geopy.distance import great_circle
import pandas as pd
import folium

class CountByWGS84:

    def __init__(self, df, lat, lon, dist=1):
        """
        df: 데이터 프레임
        lat: 중심 위도
        lon: 중심 경도
        dist: 기준 거리(km)
        """
        self.df = df
        self.lat = lat
        self.lon = lon
        self.dist = dist

    def filter_by_rectangle(self):
        """
        사각 범위 내 데이터 필터링
        """
        lat_min = self.lat - 0.01 * self.dist
        lat_max = self.lat + 0.01 * self.dist

        lon_min = self.lon - 0.015 * self.dist
        lon_max = self.lon + 0.015 * self.dist

        self.points = [[lat_min, lon_min], [lat_max, lon_max]]

        result = self.df.loc[
            (self.df['lat'] > lat_min) &
            (self.df['lat'] < lat_max) &
            (self.df['lon'] > lon_min) &
            (self.df['lon'] < lon_max)
        ]
        result.index = range(len(result))

        return result

    def filter_by_radius(self):
        """
        반경 범위 내 데이터 필터링
        """
        # 사각 범위 내 데이터 필터링
        tmp = self.filter_by_rectangle()

        # 기준 좌표 포인트
        center = (self.lat, self.lon)

        result = pd.DataFrame()
        for index, row in tmp.iterrows():
            # 개별 좌표 포인트
            point = (row['lat'], row['lon'])
            d = great_circle(center, point).kilometers
            if d <= self.dist:
                result = pd.concat([result, tmp.iloc[index, :].to_frame().T])

        result.index = range(len(result))

        return result

    def plot_by_rectangle(self, df):
        """
        사각 범위 내 데이터 플로팅
        """

        m = folium.Map(location=[self.lat, self.lon], zoom_start=14)

        for idx, row in df.iterrows():

            lat_ = row['lat']
            lon_ = row['lon']

            folium.Marker(location=[lat_, lon_],
                          radius=15,
                          tooltip=row['지점명']).add_to(m)

        folium.Rectangle(bounds=self.points,
                         color='#ff7800',
                         fill=True,
                         fill_color='#ffff00',
                         fill_opacity=0.2).add_to(m)

        return m

    def plot_by_radius(self, df):
        """
        반경 범위 내 데이터 플로팅
        """

        m = folium.Map(location=[self.lat, self.lon], zoom_start=14)

        for idx, row in df.iterrows():

            lat_ = row['lat']
            lon_ = row['lon']

            folium.Marker(location=[lat_, lon_],
                          radius=15,
                          tooltip=row['lat']).add_to(m)

        folium.Circle(radius=dist * 1000,
                      location=[lat, lon],
                      color="#ff7800",
                      fill_color='#ffff00',
                      fill_opacity=0.2
                      ).add_to(m)

        return m
        return m

## 위경도 정보가 포함된 데이터 불러오기

In [4]:
# Sample Dataset
df = pd.read_csv("./교통사고_종로구.csv")

In [5]:
# df = df.loc[df['상호명'].str.contains("스타벅스", na=False)]
df = df.rename(columns={'경도': 'lon', '위도': 'lat'})
df.index = range(len(df))

In [6]:
df.head()

,Unnamed: 0,년도,월,일,요일,시각,사고등급,사망자수,중상자수,경상자수,...,기상상태,일광상태,사고유형1,사고유형2,법규위반,도로종류,지점명,lat,lon,자치구
0,0,2017,7,13,목요일,9,부상사고,0,0,0,...,맑음,주간,차대차,측면직각충돌,안전거리미확보,특별광역시도,관철동14-1번지사거리,37.570176,126.983197,종로구
1,1,2017,9,1,금요일,3,중상사고,0,1,0,...,맑음,심야,차대차,차대차-기타,안전운전불이행,특별광역시도,관철동14-1번지사거리,37.570176,126.983197,종로구
2,2,2017,11,24,금요일,1,경상사고,0,0,1,...,맑음,심야,차대사람,횡단중,신호위반,특별광역시도,관철동14-1번지사거리,37.570176,126.983197,종로구
3,3,2018,3,24,토요일,20,경상사고,0,0,1,...,맑음,출퇴근,차대사람,차도통행중,안전운전불이행,특별광역시도,관철동14-1번지사거리,37.570176,126.983197,종로구
4,4,2018,7,28,토요일,18,경상사고,0,0,2,...,맑음,출퇴근,차대차,차대차-기타,안전거리미확보,특별광역시도,관철동14-1번지사거리,37.570176,126.983197,종로구


In [7]:
df['lat']

0      37.570176
1      37.570176
2      37.570176
3      37.570176
4      37.570176
         ...    
838    37.585533
839    37.585533
840    37.585533
841    37.585533
842    37.585533
Name: lat, Length: 843, dtype: float64

In [8]:
df=df[['lat','lon']]

In [9]:
lat = 37.570033541783
lon = 126.98167021854
dist = 10

In [10]:
cbw = CountByWGS84(df, lat, lon, dist)

In [11]:
# 사각 범위 내 데이터 필터링
result_rectangle = cbw.filter_by_rectangle()

# 반경 범위 내 데이터 필터링
result_radius = cbw.filter_by_radius()

print(f"""
{"="*50}
중심 위도: {cbw.lat}
중심 경도: {cbw.lon}
기준 거리: {cbw.dist} km
사각 범위 내 데이터 필터링 결과: {len(result_rectangle):,} 건
반경 범위 내 데이터 필터링 결과: {len(result_radius):,} 건
{"="*50}
""")


중심 위도: 37.570033541783
중심 경도: 126.98167021854
기준 거리: 10 km
사각 범위 내 데이터 필터링 결과: 843 건
반경 범위 내 데이터 필터링 결과: 843 건



In [12]:
plot_2 = cbw.plot_by_radius(result_radius)
plot_2

In [1]:
# cbw.plot_by_radius()